In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd '/content/gdrive/MyDrive/Colab Notebooks/Modified-IRNet'

/content/gdrive/MyDrive/Colab Notebooks/Modified-IRNet


In [ ]:
pwd

'/content/gdrive/MyDrive/Colab Notebooks/Modified-IRNet'

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4') ##SS: colab

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
pip install pattern

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import time
import traceback

In [ ]:
import os
import torch
import torch.optim as optim
import tqdm
import copy

In [ ]:
from src import args as arg
from src import utils
from src.models.model import IRNet
from src.rule import semQL

In [ ]:
def train(args):
    """
    :param args:
    :return:
    """

    grammar = semQL.Grammar()
    sql_data, table_data, val_sql_data,\
    val_table_data= utils.load_dataset(args.dataset, use_small=args.toy)

    model = IRNet(args, grammar)


    if args.cuda: model.cuda()

    # now get the optimizer
    optimizer_cls = eval('torch.optim.%s' % args.optimizer)
    optimizer = optimizer_cls(model.parameters(), lr=args.lr)
    print('Enable Learning Rate Scheduler: ', args.lr_scheduler)
    if args.lr_scheduler:
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[21, 41], gamma=args.lr_scheduler_gammar)
    else:
        scheduler = None

    print('Loss epoch threshold: %d' % args.loss_epoch_threshold)
    print('Sketch loss coefficient: %f' % args.sketch_loss_coefficient)

    if args.load_model:
        print('load pretrained model from %s'% (args.load_model))
        pretrained_model = torch.load(args.load_model,
                                         map_location=lambda storage, loc: storage)
        pretrained_modeled = copy.deepcopy(pretrained_model)
        for k in pretrained_model.keys():
            if k not in model.state_dict().keys():
                del pretrained_modeled[k]

        model.load_state_dict(pretrained_modeled)

    model.word_emb = utils.load_word_emb(args.glove_embed_path)
    # begin train

    model_save_path = utils.init_log_checkpoint_path(args)
    #utils.save_args(args, os.path.join(model_save_path, 'config.json')) #SS
    best_dev_acc = .0

    try:
        with open(os.path.join(model_save_path, 'epoch.log'), 'w') as epoch_fd:
            for epoch in tqdm.tqdm(range(args.epoch)):
                if args.lr_scheduler:
                    scheduler.step()
                epoch_begin = time.time()
                loss = utils.epoch_train(model, optimizer, args.batch_size,
                                   sql_data, table_data, args,
                                   loss_epoch_threshold=args.loss_epoch_threshold,
                                   sketch_loss_coefficient=args.sketch_loss_coefficient)
                epoch_end = time.time()
                #log_str = 'Epoch: %d, Loss: %f, Training Sketch Acc: %f, Training Acc: %f, time: %f\n' % (
                #    epoch + 1, loss, sketch_acc, acc, epoch_end - epoch_begin)
                json_datas, sketch_acc, acc = utils.epoch_acc(model, args.batch_size, val_sql_data, val_table_data,
                                             beam_size=args.beam_size)
                eval_acc, nest_acc = utils.eval_acc(json_datas, val_sql_data)

                if acc > best_dev_acc:
                #if nest_acc > best_dev_acc:
                    utils.save_checkpoint(model, os.path.join(model_save_path, 'best_model.model'))
                    #utils.save_checkpoint(model, os.path.join(model_save_path, 'best_nest_model.model'))
                    best_dev_acc = acc
                utils.save_checkpoint(model, os.path.join(model_save_path, '{%s}_{%s}.model') % (epoch, acc))

                log_str = 'Epoch: %d, Loss: %f, Val Sketch Acc: %f, Val Acc: %f : %f, Nested Acc: %f, time: %f\n' % (
                    epoch + 1, loss, sketch_acc, acc, eval_acc, nest_acc, epoch_end - epoch_begin)
                tqdm.tqdm.write(log_str)
                epoch_fd.write(log_str)
                epoch_fd.flush()
    except Exception as e:
        # Save model
        utils.save_checkpoint(model, os.path.join(model_save_path, 'end_model.model'))
        print(e)
        tb = traceback.format_exc()
        print(tb)
    else:
        utils.save_checkpoint(model, os.path.join(model_save_path, 'end_model.model'))
        json_datas, sketch_acc, acc = utils.epoch_acc(model, args.batch_size, val_sql_data, val_table_data,
                                     beam_size=args.beam_size)
        # acc = utils.eval_acc(json_datas, val_sql_data)

        print("Validation Sketch Acc: %f, Validation Acc: %f, Validation Beam Acc: %f" % (sketch_acc, acc, acc,))

In [ ]:
## Arguments
class args:
    dataset = './data'
    glove_embed_path = './data/glove.42B.300d.txt'
    cuda = True
    epoch = 50 ##ss: 50
    loss_epoch_threshold = 50
    sketch_loss_coefficient = 0.85
    beam_size = 1
    seed = 90
    save = './'
    embed_size = 300
    sentence_features = True
    column_pointer = True
    hidden_size = 300
    lr_scheduler = True
    lr_scheduler_gammar = 0.5
    att_vec_size = 300
    toy=False
    model_name = 'modified_irnet'
    lstm = 'lstm'
    load_model = None
    batch_size = 64
    col_embed_size = 300
    action_embed_size = 128
    type_embed_size = 128
    dropout = 0.3
    word_dropout = 0.2
    no_query_vec_to_action_map = False
    readout = 'identity'
    query_vec_to_action_diff_map = False
    column_att = 'affine'
    decode_max_time_step = 40
    save_to = 'model'
    clip_grad = 5.
    max_epoch = -1
    optimizer = 'Adam'
    lr = 0.001

In [ ]:
args.dataset

'./data'

In [ ]:
import random
import numpy as np

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)
np.random.seed(int(args.seed * 13 / 7))
random.seed(int(args.seed))

In [ ]:
'''
file_name = args.glove_embed_path
print(file_name)
with open(file_name) as inf: #SS encoding="utf-8"
    for idx, line in enumerate(inf):
        print(idx, line)
'''

'\nfile_name = args.glove_embed_path\nprint(file_name)\nwith open(file_name) as inf: #SS encoding="utf-8"\n    for idx, line in enumerate(inf):\n        print(idx, line)\n'

In [ ]:
train(args)

Loading from datasets...
Loading data from ./data/tables.json
Loading data from ./data/train.json
Loading data from ./data/dev.json
Use Column Pointer:  True
Enable Learning Rate Scheduler:  True
Loss epoch threshold: 50
Sketch loss coefficient: 0.850000
Loading word embedding from ./data/glove.42B.300d.txt
./data/glove.42B.300d.txt


  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1949: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecat

0.20485436893203884 0.08333333333333333
Epoch: 1, Loss: 10.644575, Val Sketch Acc: 0.619417, Val Acc: 0.274757 : 0.204854, Nested Acc: 0.083333, time: 85.850069



  4%|▍         | 2/50 [04:17<1:43:40, 129.60s/it]

0.28640776699029125 0.1794871794871795
Epoch: 2, Loss: 4.295348, Val Sketch Acc: 0.703883, Val Acc: 0.376699 : 0.286408, Nested Acc: 0.179487, time: 94.328762



...

 98%|█████████▊| 49/50 [5:11:42<10:23, 623.07s/it]

0.36699029126213595 0.24358974358974358
Epoch: 49, Loss: 0.025830, Val Sketch Acc: 0.803883, Val Acc: 0.492233 : 0.366990, Nested Acc: 0.243590, time: 576.300810



100%|██████████| 50/50 [5:22:27<00:00, 386.95s/it]

0.3640776699029126 0.24358974358974358
Epoch: 50, Loss: 0.026702, Val Sketch Acc: 0.790291, Val Acc: 0.487379 : 0.364078, Nested Acc: 0.243590, time: 559.147318



Validation Sketch Acc: 0.790291, Validation Acc: 0.487379, Validation Beam Acc: 0.487379
